In [1]:
%run -i '../../../utils/setup_environment.ipy'

import warnings
warnings.filterwarnings('ignore')
from scipy.stats import fisher_exact, ranksums, chi2, norm
from statsmodels.sandbox.stats.multicomp import multipletests
import matplotlib.gridspec as gridspec

data_path = '../../../data/'

Setup environment... done!


<span style="color:green">✅ Working on **mskimpact_env** conda environment.</span>

In [2]:
arm_level = pd.read_csv(data_path + 'impact-facets-tp53/new_data_facets/msk_impact_facets_annotated.arm_level.txt', sep='\t')
gene_level = pd.read_csv(data_path + 'impact-facets-tp53/new_data_facets/msk_impact_facets_annotated.gene_level_grepTP53.txt', sep='\t')

In [4]:
# TO CHECK
# Inconsistencies gene_level - arm_level in tp53
from tqdm import tqdm,tqdm_notebook
tqdm_notebook().pandas()
def inconsistency(x):
    if arm_level[arm_level['sample'] == x['sample']][arm_level['arm'] == '17p']['cn_state'].shape[0]>0:
        if arm_level[arm_level['sample'] == x['sample']][arm_level['arm'] == '17p']['cn_state'].values[0] != x.cn_state:
            return True
        else: return False
    else: return False

    

gene_level_tp53 = gene_level[gene_level['gene'] == 'TP53']
gene_level_tp53['inconcistency'] = gene_level_tp53.progress_apply(inconsistency, axis=1)

In [9]:
gene_level_tp53 = gene_level_tp53.rename(columns={"inconcistency": "inconsistency"})

In [29]:
import pickle
with open('samples_cohort.pkl', 'rb') as f:
    samples_cohort = pickle.load(f)
    
gene_level_tp53_cohort = gene_level_tp53[gene_level_tp53['sample'].str[:17].isin(samples_cohort)]

In [55]:
get_groupby(gene_level_tp53, 'inconsistency', 'count')

,count
inconsistency,
False,34807
True,7409


In [56]:
get_groupby(gene_level_tp53[gene_level_tp53['inconsistency'] == True], 'cn_state', 'count').sort_values(by='count', ascending=False)

,count
cn_state,
INDETERMINATE,3014
HETLOSS,918
DIPLOID,572
LOSS BEFORE,383
CNLOH,355
CNLOH BEFORE & LOSS,345
GAIN,324
HOMDEL,291
LOSS AFTER,257


In [78]:
samples_hetloss = set(list(gene_level_tp53[gene_level_tp53['inconsistency'] == True][gene_level_tp53['cn_state'] == 'DIPLOID']['sample']))

get_groupby(arm_level[arm_level['sample'].isin(samples_hetloss)][arm_level['arm'] == '17p'], 'cn_state', 'count')
            
            

,count
cn_state,
AMP,1
CNLOH,84
CNLOH & GAIN,4
CNLOH BEFORE & LOSS,1
DOUBLE LOSS AFTER,93
GAIN,107
HETLOSS,212
HOMDEL,39
LOSS & GAIN,1


In [79]:
samples_hetloss = set(list(gene_level_tp53[gene_level_tp53['inconsistency'] == True][gene_level_tp53['cn_state'] == 'DIPLOID']['sample']))
samples_het_dip = list(set(arm_level[arm_level['sample'].isin(samples_hetloss)][arm_level['arm'] == '17p'][arm_level['cn_state'] == 'HETLOSS']['sample']))

seg_length = gene_level_tp53[gene_level_tp53['sample'].isin(samples_het_dip)]['seg_length']

pd.set_option('float_format', '{:f}'.format)
pd.DataFrame(seg_length.describe())


,seg_length
count,212.000000
mean,48464205.245283
std,23573654.321111
min,8268530.000000
25%,29567128.750000
50%,38425107.000000
75%,80668371.500000
max,80668800.000000


In [46]:
arm_level[arm_level['sample'] =='P-0009819-T01-IM5_P-0009819-N01-IM5'][arm_level['arm'] =='17p']

,sample,arm,tcn,lcn,cn_length,arm_length,frac_of_arm,cn_state
67,P-0009819-T01-IM5_P-0009819-N01-IM5,17p,2,1,22148282,22148282,1.0,DIPLOID


In [38]:
annotated_cohort = pd.read_csv(data_path + 'impact-facets-tp53/new_data_facets/msk_impact_facets_annotated.cohort.txt', sep='\t')

In [44]:
annotated_cohort[annotated_cohort['sample_id'] == 'P-0049537-T01-IM6_P-0049537-N01-IM6']['wgd'].values[0]

True

In [68]:
gene_level_tp53

,sample,gene,chrom,gene_start,gene_end,tsg,seg,median_cnlr_seg,segclust,seg_start,seg_end,cf.em,tcn.em,lcn.em,cf,tcn,lcn,seg_length,mcn,genes_on_seg,gene_snps,gene_het_snps,spans_segs,cn_state,filter,inconsistency
0,P-0034223-T01-IM6_P-0034223-N01-IM6,TP53,17,7565097,7590868,True,26,-0.178258,5,114669,80783261,1.000000,2,1.0,1.000000,2,1.0,80668592,1.0,1203,27,0,False,DIPLOID,PASS,False
2,P-0009819-T01-IM5_P-0009819-N01-IM5,TP53,17,7565097,7590868,True,21,-0.237684,6,114724,80783024,1.000000,2,1.0,1.000000,2,1.0,80668300,1.0,1203,20,4,False,DIPLOID,PASS,False
3,P-0025956-T01-IM6_P-0025956-N01-IM6,TP53,17,7565097,7590868,True,25,-0.050441,4,114669,80783100,0.169170,3,1.0,0.169170,3,1.0,80668431,2.0,1203,20,0,False,LOSS AFTER,PASS,False
5,P-0027609-T01-IM6_P-0027609-N01-IM6,TP53,17,7565097,7590868,True,27,-0.135182,2,114500,18575692,1.000000,2,1.0,1.000000,2,1.0,18461192,1.0,317,21,4,False,DIPLOID,PASS,False
7,P-0027408-T01-IM6_P-0027408-N01-IM6,TP53,17,7565097,7590868,True,30,-0.130530,6,114450,26370475,0.275073,1,0.0,0.275073,1,0.0,26256025,1.0,359,21,4,False,HETLOSS,PASS,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71757,P-0047736-T01-IM6_P-0047736-N01-IM6,TP53,17,7565097,7590868,True,27,-0.374636,6,114669,80783217,0.184186,2,0.0,0.184186,2,0.0,80668548,2.0,1203,26,5,False,CNLOH,PASS,False
71759,P-0049537-T01-IM6_P-0049537-N01-IM6,TP53,17,7565097,7590868,True,47,-0.493030,5,114629,21544493,0.632827,1,0.0,0.632827,1,0.0,21429864,1.0,349,24,0,False,HETLOSS,PASS,True
71761,P-0047148-T01-IM6_P-0047148-N01-IM6,TP53,17,7565097,7590868,True,28,-0.118122,12,114700,80783100,1.000000,2,1.0,1.000000,2,1.0,80668400,1.0,1203,24,3,False,DIPLOID,PASS,False
71763,P-0049188-T01-IM6_P-0049188-N01-IM6,TP53,17,7565097,7590868,True,55,0.111608,16,114530,7990700,NaN,2,NaN,NaN,2,NaN,7876170,NaN,205,26,1,False,INDETERMINATE,PASS,True
